In [3]:
import MetaTrader5 as mt5
import pandas as pd

In [5]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

In [20]:
ativo = 'WDOF23'
df1 = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 9100))
df1['data'] = pd.to_datetime(df1['time'], unit='s')
df1

,time,open,high,low,close,tick_volume,spread,real_volume,data
0,1669398120,5431.0,5431.0,5431.0,5431.0,1,15,2,2022-11-25 17:42:00
1,1669398180,5434.0,5434.0,5434.0,5434.0,9,5,30,2022-11-25 17:43:00
2,1669398240,5434.5,5434.5,5434.5,5434.5,1,10,2,2022-11-25 17:44:00
3,1669398540,5436.5,5436.5,5436.0,5436.0,3,5,3,2022-11-25 17:49:00
4,1669398720,5435.5,5435.5,5435.5,5435.5,1,10,1,2022-11-25 17:52:00
...,...,...,...,...,...,...,...,...,...
9095,1671474300,5307.5,5309.0,5304.5,5308.5,656,5,2808,2022-12-19 18:25:00
9096,1671474360,5309.0,5310.0,5307.5,5309.0,245,5,1184,2022-12-19 18:26:00
9097,1671474420,5309.0,5309.5,5305.5,5306.5,138,5,613,2022-12-19 18:27:00
9098,1671474480,5306.5,5307.0,5305.0,5305.5,156,5,515,2022-12-19 18:28:00


In [12]:
df1

,time,open,high,low,close,tick_volume,spread,real_volume,data
0,1671189540,5300.5,5300.5,5296.0,5297.5,2679,5,6996,2022-12-16 11:19:00
1,1671189600,5297.5,5299.5,5295.5,5299.0,2353,5,6820,2022-12-16 11:20:00
2,1671189660,5299.0,5300.5,5298.5,5300.0,1509,5,3742,2022-12-16 11:21:00
3,1671189720,5300.0,5300.5,5297.0,5299.0,1856,5,6094,2022-12-16 11:22:00
4,1671189780,5299.0,5302.0,5298.0,5301.5,2205,5,7243,2022-12-16 11:23:00
...,...,...,...,...,...,...,...,...,...
994,1671474300,5307.5,5309.0,5304.5,5308.5,656,5,2808,2022-12-19 18:25:00
995,1671474360,5309.0,5310.0,5307.5,5309.0,245,5,1184,2022-12-19 18:26:00
996,1671474420,5309.0,5309.5,5305.5,5306.5,138,5,613,2022-12-19 18:27:00
997,1671474480,5306.5,5307.0,5305.0,5305.5,156,5,515,2022-12-19 18:28:00
